In [1]:
import os

print(os.path.abspath(os.curdir))

e:\Data Kuliah\S2 ITS\Semester 5\Sentiment Analysis\drone-severity-multitask\notebooks


In [2]:
import os
os.chdir("..")

In [3]:
print(os.path.abspath(os.curdir))

e:\Data Kuliah\S2 ITS\Semester 5\Sentiment Analysis\drone-severity-multitask


In [4]:
import json
import logging
import sys
import pandas as pd
from os.path import dirname

from drain3 import TemplateMiner
from drain3.template_miner_config import TemplateMinerConfig
# For more mechanical-based baselines
# https://github.com/logpai/logparser

# persistence_type = "NONE"
# persistence_type = "REDIS"
# persistence_type = "KAFKA"
persistence_type = "FILE"
filtered = pd.read_csv('dataset/merged-manual-unique.csv')

logger = logging.getLogger(__name__)
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format='%(message)s')

if persistence_type == "KAFKA":
    from drain3.kafka_persistence import KafkaPersistence

    persistence = KafkaPersistence("drain3_state", bootstrap_servers="localhost:9092")

elif persistence_type == "FILE":
    from drain3.file_persistence import FilePersistence

    persistence = FilePersistence("drain3_state.bin")

elif persistence_type == "REDIS":
    from drain3.redis_persistence import RedisPersistence

    persistence = RedisPersistence(redis_host='',
                                   redis_port=25061,
                                   redis_db=0,
                                   redis_pass='',
                                   is_ssl=True,
                                   redis_key="drain3_state_key")
else:
    persistence = None
persistence = None
config = TemplateMinerConfig()
config.load(f"notebooks/drain3.ini")
config.profiling_enabled = False

template_miner = TemplateMiner(persistence, config)
print(f"Drain3 started with '{persistence_type}' persistence")
print(f"{len(config.masking_instructions)} masking instructions are in use")
print(f"Starting training mode. Reading from std-in ('q' to finish)")
for ind in filtered.index:
    # print(filtered['message'][ind], filtered['label'][ind])

    # log_line = input("> ")
    # if log_line == 'q':
    #     break
    result = template_miner.add_log_message(filtered['message'][ind])
    result_json = json.dumps(result)
    print(result_json)
    template = result["template_mined"]
    params = template_miner.extract_parameters(template, filtered['message'][ind])
    print(f"Parameters: {str(params)}")

print("Training done. Mined clusters:")
for cluster in template_miner.drain.clusters:
    print(cluster)

Starting Drain3 template miner
Drain3 started with 'FILE' persistence
7 masking instructions are in use
Starting training mode. Reading from std-in ('q' to finish)
{"change_type": "cluster_created", "cluster_id": 1, "cluster_size": 1, "template_mined": "A passenger aircraft is approaching. Descend as soon as possible", "cluster_count": 1}
Parameters: []
{"change_type": "cluster_created", "cluster_id": 2, "cluster_size": 1, "template_mined": "A passenger aircraft is nearby. Fly with caution", "cluster_count": 2}
Parameters: []
{"change_type": "cluster_created", "cluster_id": 3, "cluster_size": 1, "template_mined": "Abnormal compass function or GPS signal detected. Aircraft switched to ATTI mode", "cluster_count": 3}
Parameters: []
{"change_type": "cluster_created", "cluster_id": 4, "cluster_size": 1, "template_mined": "Accelerator is Over Range", "cluster_count": 4}
Parameters: []
{"change_type": "cluster_created", "cluster_id": 5, "cluster_size": 1, "template_mined": "Account not logge